In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import nltk
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
data = pd.read_csv("C:\\Users\\bhimr\\Downloads\\blogs (1).csv")
data.to_csv("C:\\Users\\bhimr\\Downloads\\blogs_no_header.csv", index=False, header=False)

In [3]:
data.head()

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [5]:
data.isnull().sum()

Data      0
Labels    0
dtype: int64

In [6]:
#Cleaning the Text
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation, and remove words containing numbers.'''
    import re
    import string

    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)  # Raw string to handle special characters
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)  # Removes words with numbers
    text = re.sub(r'[0-9]+', ' ', text)
    text = re.sub(r'[‘’“”…]', '', text)
    return text


In [7]:
freq = pd.Series(' '.join(data['Data']).split()).value_counts()[:20]
freq


the     23314
to      12877
of      12222
a       10312
and      9916
I        7417
is       7235
in       6962
that     6443
>        5075
for      4645
you      3855
it       3645
on       3512
be       3274
have     3096
with     3017
are      2927
not      2881
The      2576
Name: count, dtype: int64

In [8]:
#Removing Stop Words
stop = stopwords.words('english')

data['Data'] = data['Data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

freq_Sw = pd.Series(' '.join(data['Data']).split()).value_counts()[:20]
freq_Sw


I                7417
>                5075
The              2576
Subject:         2056
From:            2033
Newsgroups:      2011
Date:            2008
Path:            2003
Message-ID:      2003
Lines:           1998
Organization:    1937
Apr              1937
GMT              1785
|                1641
-                1595
In               1495
would            1485
writes:          1452
--               1422
1993             1403
Name: count, dtype: int64

In [9]:
# Initializing CountVectorizer
vectorizer = CountVectorizer(min_df=1, max_df=0.9)
X = vectorizer.fit_transform(data["Data"])
#Creating a DataFrame fro word frequencies
word_freq_df = pd.DataFrame({'term': vectorizer.get_feature_names_out(), 
                             'occurrences': np.asarray(X.sum(axis=0)).ravel().tolist()})
#Calculate word Frequencies
word_freq_df['frequency'] = word_freq_df['occurrences'] / np.sum(word_freq_df['occurrences'])
word_freq_df.head(30)


,term,occurrences,frequency
0,00,339,0.000687
1,000,103,0.000209
2,0000,5,0.000010
3,00000,1,0.000002
4,0000001200,1,0.000002
5,000003,1,0.000002
6,000005102000,2,0.000004
7,00000510200001,1,0.000002
8,00000f,1,0.000002
9,000036,1,0.000002


In [10]:
#Initializing TfidVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.5)
doc_vec = vectorizer.fit_transform(data["Data"])

df = pd.DataFrame(doc_vec.todense(), columns=vectorizer.get_feature_names_out())
df


,00,000,01,02,03,04,05,06,07,08,...,x11,x11r5,xref,yale,year,years,yes,york,young,zaphod
0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.036175,0.0,0.0,0.018811
1,0.000000,0.000000,0.0,0.120197,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.070080
2,0.000000,0.000000,0.0,0.063697,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
3,0.000000,0.026673,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.020147,0.018728,0.133085,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.049562,...,0.0,0.0,0.030010,0.0,0.000000,0.041513,0.000000,0.0,0.0,0.025566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.019009,0.0,0.000000,0.026296,0.062288,0.0,0.0,0.016194
1996,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.034911,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1997,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.076426,0.000000,...,0.0,0.0,0.045707,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.038939
1998,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.043446,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [11]:
#Extracting top n words from the word corpus
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2, 2), max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec1.vocabulary_.items()]
    return sorted(words_freq, key=lambda x: x[1], reverse=True)[:n]

top2_df = pd.DataFrame(get_top_n2_words(data["Data"], n=200), columns=["Bi-gram", "Freq"])

top2_df.head()


,Bi-gram,Freq
0,cmu edu,5206
1,cs cmu,3250
2,srv cs,3185
3,cantaloupe srv,2574
4,message id,2007


In [ ]:
#Displaying the word Count
wordcloud_stw = WordCloud(background_color='black', width=1800, height=1500).generate(" ".join(data["Data"]))
plt.imshow(wordcloud_stw)

In [ ]:
blog_train, blog_test = train_test_split(data, test_size=0.3)



In [ ]:
# Define the split_into_words function
def split_into_words(i):
    return (i.split(" "))

# Now use it in CountVectorizer
blog_bow = CountVectorizer(analyzer=split_into_words).fit(data.Data)
train_blog_matrix = blog_bow.transform(blog_train.Data)
test_blog_matrix = blog_bow.transform(blog_test.Data)


In [ ]:
classifier_mb = MB()
classifier_mb.fit(train_blog_matrix, blog_train.Labels)
test_pred_m = classifier_mb.predict(test_blog_matrix)
accuracy_test_m = np.mean(test_pred_m == blog_test.Labels)

In [ ]:
print(test_pred_m)



In [ ]:
print(accuracy_test_m)

In [ ]:
# Train Gaussian Naive Bayes model
classifier_gb = GB()
classifier_gb.fit(train_blog_matrix.toarray(), blog_train.Labels.values)
# Predict on the test data
test_pred_g = classifier_gb.predict(test_blog_matrix.toarray())
# Evaluate accuracy
accuracy_test_g = np.mean(test_pred_g == blog_test.Labels)
print("Test Accuracy (Gaussian Naive Bayes):", accuracy_test_g)

In [ ]:
#!pip install textblob


In [ ]:
# Function to classify sentiment
def get_sentiment(text):
    '''Returns 'Positive', 'Negative', or 'Neutral' based on the sentiment polarity of the text.'''
    sentiment = TextBlob(text).sentiment.polarity  # Polarity ranges from -1 (negative) to 1 (positive)
    if sentiment > 0:
        return 'Positive'
    elif sentiment < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment classification
data['sentiment'] = data['Data'].apply(get_sentiment)

# Show the first few rows with the sentiment classification
print(data[['Data', 'sentiment']].head())


In [ ]:
# Visualize sentiment distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='sentiment', data=data, palette='viridis')
plt.title('Sentiment Distribution in Emails')
plt.show()

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(blog_test.Labels, test_pred_m)
# Plot confusion matrix as a heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=blog_test['Labels'].unique(), yticklabels=blog_test['Labels'].unique())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
print(cm)